In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
266,AFG,Asia,Afghanistan,2020-09-22,39074.0,30.0,37.000,1444.0,3.0,2.571,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
464,ALB,Europe,Albania,2020-09-22,12535.0,150.0,145.000,364.0,2.0,3.714,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
731,DZA,Africa,Algeria,2020-09-22,50023.0,197.0,218.143,1679.0,7.0,8.429,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-21,France,466143.0,0.0,31324.0,0.0,0.0,0.0
0,2020-09-21,China,90399.0,0.0,4742.0,0.0,0.0,0.0
0,2020-09-21,Italy,299592.0,0.0,35746.0,0.0,0.0,0.0
0,2020-09-21,Spain,643901.0,0.0,30502.0,0.0,0.0,0.0
0,2020-09-21,United States,6881428.0,0.0,200703.0,0.0,0.0,0.0
0,2020-09-21,World,31625968.0,0.0,971120.0,0.0,0.0,0.0
0,2020-09-21,United Kingdom,397604.0,0.0,41830.0,0.0,0.0,0.0
0,2020-09-21,Germany,273609.0,0.0,9393.0,0.0,0.0,0.0
0,2020-09-21,Iran,425291.0,0.0,24507.0,0.0,0.0,0.0
0,2020-09-21,Turkey,304102.0,0.0,7561.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-22,France,466143.0,458061.0,31324.0,31338.0,8082.0,-14.0
0,2020-09-22,China,90399.0,90381.0,4742.0,4737.0,18.0,5.0
0,2020-09-22,Italy,299592.0,299506.0,35746.0,35724.0,86.0,22.0
0,2020-09-21,Spain,643901.0,671468.0,30502.0,30663.0,-27567.0,-161.0
0,2020-09-22,United States,6881428.0,6857967.0,200703.0,199881.0,23461.0,822.0
0,2020-09-22,World,31625968.0,31377058.0,971120.0,965681.0,248910.0,5439.0
0,2020-09-22,United Kingdom,397604.0,398625.0,41830.0,41788.0,-1021.0,42.0
0,2020-09-22,Germany,273609.0,274158.0,9393.0,9396.0,-549.0,-3.0
0,2020-09-22,Iran,425291.0,425481.0,24507.0,24478.0,-190.0,29.0
0,2020-09-22,Turkey,304102.0,304610.0,7561.0,7574.0,-508.0,-13.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")